In [ ]:
!pip install pyyaml==5.1
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [3]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
torch:  1.11 ; cuda:  cu113
detectron2: 0.6


In [4]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import json
from pandas.io.json import json_normalize
import os
import torch
import copy

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader, build_detection_train_loader
from detectron2.data import detection_utils as utils
import detectron2.data.transforms as T

from pycocotools.coco import COCO
import skimage.io as io
from detectron2.utils.visualizer import ColorMode

from PIL import Image

In [ ]:
!pip uninstall albumentations
!pip install albumentations==1.1.0
!pip install "opencv-python-headless<4.3"

In [ ]:
import albumentations as A
print(A.__version__)#1.1.0

###  transform 정의

In [ ]:
import albumentations as A
import numpy as np
import cv2
from PIL import Image
import random
random.seed(57)


train_transform_1 = A.Compose([
                             A.OneOf([
                                      A.HorizontalFlip(p=1),
                                      A.Rotate(limit=30, p=1),
                                      A.VerticalFlip(p=1),
                                      A.ColorJitter(p=1)                             

                             ])]) # transform 기법 바꿔보기


train_transform_2 = A.Compose([
                            A.OneOf([
                                      A.HorizontalFlip(p=1),
                                      A.RandomRotate90(p=1), # 완전 뒤집힘 말고 45정도까지
                                      A.VerticalFlip(p=1)], p=1),
                            A.OneOf([
                                    A.GaussNoise(p=1),
                                    A.RandomBrightness(p=1),
                                    A.Blur(p=1)], p=0.7) # 블러 limit 낮추기
])


train_transform_3 = A.Compose([
                             A.OneOf([
                                      A.HorizontalFlip(p=1),
                                      A.RandomRotate90(p=1),
                                      A.VerticalFlip(p=1),
                                      ], p=1),]) # position 증강

train_transform_4 = A.Compose([
                             A.Blur(blur_limit=[5,5], p=1)]) # blur 처리

train_transfrom_5 =  A.Compose([
                             A.OneOf([
                                      A.Rotate(limit = [-45,45],p=1),
                                      A.VerticalFlip(p=1),
                                      ], p=1)
                             ])

train_transfrom_6 =  A.Compose([
                             A.OneOf([
                                      A.Rotate(limit = [-45,45],p=1),
                                      A.HorizontalFlip(p=1),
                                      ], p=1)
                             ])

### 원본 파일 불러오기

In [ ]:
import glob
import os

label_img_lst = glob.glob("/개인정보 블라인드/train/labeled_images/"+"*.jpg")
mask_lst = glob.glob("/개인정보 블라인드/train/labels/"+"*.png")

label_img_lst.sort()
mask_lst.sort()

In [ ]:
len(label_img_lst), len(mask_lst)

(135, 135)

### 각 클래스 갯수 확인

In [ ]:
container = 0
forklift = 0
reach_stacker = 0
ship = 0

# 57은 이미지 경로 length, -15는 이미지 파일 이름 length
for idx, name in enumerate(label_img_lst):
  if label_img_lst[idx][57:-15] == 'container_truck':
    container += 1
  elif label_img_lst[idx][57:-15] == 'forklift':
    forklift += 1
  elif label_img_lst[idx][57:-15] == 'reach_stacker':
    reach_stacker += 1
  elif label_img_lst[idx][57:-15] == 'ship':
    ship += 1

In [ ]:
# 원래 갯수
print(container , forklift, reach_stacker,ship)

32 37 13 53


## Function 정의

In [ ]:
# 이미지 불러와서 트랜스폼 버전에 맞게 이미지와 마스크를 변환 후 반환하는 함수

def image_transform(item, mask, transfrom_version):
  im = cv2.imread(item)
  im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
  aug_mask = np.asarray(Image.open(mask))

  transformed = transfrom_version(image=im, mask=aug_mask) 
  transformed_image = transformed['image']
  transformed_mask = transformed['mask']
  
  return transformed_image,transformed_mask

In [ ]:
import skimage.measure as measure

def close_contour(contour):
  if not np.array_equal(contour[0], contour[-1]):
    contour = np.vstack((contour, contour[0]))
  return contour

def binary_mask_to_polygon(binary_mask, tolerance=0):
  polygons = []
  # pad mask to close contours of shapes which start and end at an edge
  padded_binary_mask = np.pad(binary_mask, pad_width=1, mode='constant', constant_values=0)
  contours = measure.find_contours(padded_binary_mask, 0.5)
  contours = np.subtract(contours, 1)
  for contour in contours:
      contour = close_contour(contour)
      contour = measure.approximate_polygon(contour, tolerance)
      if len(contour) < 3: 
          continue
      contour = np.flip(contour, axis=1)
      segmentation = contour.ravel().tolist()
      # after padding and subtracting 1 we may get -0.5 points in our segmentation
      segmentation = [0 if i < 0 else i for i in segmentation]
      polygons.append(segmentation)

  return polygons

In [ ]:
classes = ['container_truck', 'forklift', 'reach_stacker', 'ship']

In [ ]:
aug_img_lst = []
aug_mask_lst = []
sample_preds_folder = "/개인정보 블라인드/"

# 증강 후 저장
for item, mask in zip(label_img_lst,mask_lst):

  if item[57:-15] in ['forklift', 'container_truck']:
    transformed_image,transformed_mask = image_transform(item, mask,train_transform_1) # transform 버전 바꾸기!!!!!!!!!!!!!!!!!!!!!!
    aug_img_lst.append(transformed_image) #확인용
    aug_mask_lst.append(transformed_mask)

    cv2.imwrite(os.path.join(sample_preds_folder,"random_img/","{}{}.jpg".format(item[57:-5],i)),transformed_image) # 새로운 이미지 저장
    cv2.imwrite(os.path.join(sample_preds_folder,"random_mask/","{}{}.png".format(item[57:-5],i)), transformed_mask)

  if item[57:-15] == 'reach_stacker':
    for i in range(2):
      transformed_image,transformed_mask = image_transform(item, mask,train_transform_1) # transform 버전 바꾸기!!!!!!!!!!!!!!!!!!!!!!
      aug_img_lst.append(transformed_image) #확인용
      aug_mask_lst.append(transformed_mask)

      cv2.imwrite(os.path.join(sample_preds_folder,"random_img/","{}{}.jpg".format(item[57:-5],i)),transformed_image) # 새로운 이미지 저장
      cv2.imwrite(os.path.join(sample_preds_folder,"random_mask/","{}{}.png".format(item[57:-5],i)), transformed_mask)


In [ ]:
# 위에서 저장된 파일 불러오기
label_img_lst = glob.glob(f"/개인정보 블라인드/random_img/"+"*.jpg")
mask_lst = glob.glob(f"/개인정보 블라인드/random_mask/"+"*.png")

label_img_lst.sort()
mask_lst.sort()

container = 0
forklift = 0
reach_stacker = 0
ship = 0

# 개수 세기 
for idx, name in enumerate(label_img_lst):
  if label_img_lst[idx][-30:-15] == 'container_truck':
    container += 1
  elif label_img_lst[idx][-23:-15] == 'forklift':
    forklift += 1
  elif label_img_lst[idx][-28:-15] == 'reach_stacker':
    reach_stacker += 1
  elif label_img_lst[idx][-19:-15] == 'ship':
    ship += 1

print(container,forklift,  reach_stacker, ship)
print(container+forklift+reach_stacker+ship)

32 37 26 0
95


In [ ]:
import os
import numpy as np
import json
from detectron2.structures import BoxMode



num = len("/개인정보 블라인드/random_img/")

# 랜덤으로 변환된 이미지를 피클 파일로 저장하기 위함

def get_port_dicts(file_name,label_name):

    dataset_dicts = []
    for filename, labelname in zip(file_name,label_name):

        record = {}
        objs = []

        img = Image.open(filename) #사이즈 알려고 이미지 불러옴

        record["file_name"] = filename 
        record["height"] = img.size[1] #h  
        record["width"] = img.size[0] #w

        
        mask = np.asarray(Image.open(labelname)) # 마스크 이미지 불러옴
        ins_coords = binary_mask_to_polygon(mask) # 폴리곤으로 만듦
        if len(ins_coords) != 0 :
          px = [a for a in ins_coords[0][0::2]]
          py = [a for a in ins_coords[0][1::2]]
          poly = [[x, y] for x, y in zip(px, py)]

          obj = {
            "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
            "bbox_mode": BoxMode.XYXY_ABS,
            "segmentation": [poly],
            "category_id": classes.index(filename[num:-15]), # data path에서...슬라이싱...
            "iscrowd": 0
        }

        objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)

    return dataset_dicts

In [ ]:
import glob

img_lst = glob.glob(f"/개인정보 블라인드/random_img/"+"*.jpg")
mask_lst = glob.glob(f"/개인정보 블라인드/random_mask/"+"*.png")

img_lst.sort()
mask_lst.sort()

print(len(img_lst), len(mask_lst))


In [ ]:
aug_dataset = get_port_dicts(img_lst,mask_lst)
print(len(aug_dataset))

import pickle

with open('/개인정보 블라인드/aug.pickle', 'wb') as f:
    pickle.dump(aug_dataset, f)

### Test 파일도 동일한 과정을 거침
단, mask는 존재하지 않고 img만 존재함

In [ ]:
def test_image_transform(item, transfrom_version):
  im = cv2.imread(item)
  im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

  transformed = transfrom_version(image=im) # 여기 함수 변경
  transformed_image = transformed['image']

  
  return transformed_image

In [ ]:
from tqdm import tqdm
import glob
random.seed(42)
label_img_lst = glob.glob(f"/개인정보 블라인드/test/images/"+"*.jpg")

aug_img_lst = []
sample_preds_folder = "/개인정보 블라인드/aut_test/test/"


for item in tqdm(label_img_lst):

    transformed_image = test_image_transform(item,train_transform_1)
    aug_img_lst.append(transformed_image) #확인용
   

    cv2.imwrite(os.path.join(sample_preds_folder,"{}.jpg".format(item[48:-4])),transformed_image) # 새로운 이미지 저장

print(len(aug_img_lst))